# How to read website info with _Pandas_

This notebook was created to achieve requirements of Coursera Capstone by _Vicente Carvalho_

## Objective

The main objective of this notebook is present how Pandas and Beautiful Soup libraries can be used to read website information to create a Dataframe. In our case, information is posted in https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M.

## Using _Pandas_

In [3]:
import pandas as pd
import numpy as np
pd.__version__

'0.25.1'

### Reading URL

In [19]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
web = pd.read_html(url)

In [20]:
print("The number of tables is ",len(web))

The number of tables is  3


### Checking tables received

In [21]:
df1 = web[0]
df1.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [8]:
df2 = web[1]
df2.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,NaN,Canadian postal codes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NL NS PE NB QC ON MB SK AB BC NU/NT YT A B C E...,NL NS PE NB QC ON MB SK AB BC NU/NT YT A B C E...,NL NS PE NB QC ON MB SK AB BC NU/NT YT A B C E...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NL,NS,PE,NB,QC,QC,QC,ON,ON,ON,ON,ON,MB,SK,AB,BC,NU/NT,YT
3,A,B,C,E,G,H,J,K,L,M,N,P,R,S,T,V,X,Y


In [10]:
df3 = web[2]
df3.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,NL,NS,PE,NB,QC,QC,QC,ON,ON,ON,ON,ON,MB,SK,AB,BC,NU/NT,YT
1,A,B,C,E,G,H,J,K,L,M,N,P,R,S,T,V,X,Y


Table 0 (df1) is the desired information. So let's continue only with df1.

In [27]:
df = df1.copy()

### Data Cleaning

- The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [24]:
df.columns

Index(['Postcode', 'Borough', 'Neighborhood'], dtype='object')

- Only process the cells that have an assigned borough. Ignore cells with a borough that is *Not assigned*.

In [30]:
df = df1.copy()
out = df['Borough']=="Not assigned"
for i in range(len(out)):
    if out[i]:
        df.drop(index = i, axis = 0, inplace = True)
#df1.drop(index = [], axis = 1, inplace = True)
df.reset_index(inplace = True)
df.drop(['index'], axis = 1, inplace = True)
df.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


- More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table

In [63]:
postcodes = pd.unique(df['Postcode'])
postcodes

array(['M3A', 'M4A', 'M5A', 'M6A', 'M7A', 'M9A', 'M1B', 'M3B', 'M4B',
       'M5B', 'M6B', 'M9B', 'M1C', 'M3C', 'M4C', 'M5C', 'M6C', 'M9C',
       'M1E', 'M4E', 'M5E', 'M6E', 'M1G', 'M4G', 'M5G', 'M6G', 'M1H',
       'M2H', 'M3H', 'M4H', 'M5H', 'M6H', 'M1J', 'M2J', 'M3J', 'M4J',
       'M5J', 'M6J', 'M1K', 'M2K', 'M3K', 'M4K', 'M5K', 'M6K', 'M1L',
       'M2L', 'M3L', 'M4L', 'M5L', 'M6L', 'M9L', 'M1M', 'M2M', 'M3M',
       'M4M', 'M5M', 'M6M', 'M9M', 'M1N', 'M2N', 'M3N', 'M4N', 'M5N',
       'M6N', 'M9N', 'M1P', 'M2P', 'M4P', 'M5P', 'M6P', 'M9P', 'M1R',
       'M2R', 'M4R', 'M5R', 'M6R', 'M7R', 'M9R', 'M1S', 'M4S', 'M5S',
       'M6S', 'M1T', 'M4T', 'M5T', 'M1V', 'M4V', 'M5V', 'M8V', 'M9V',
       'M1W', 'M4W', 'M5W', 'M8W', 'M9W', 'M1X', 'M4X', 'M5X', 'M8X',
       'M4Y', 'M7Y', 'M8Y', 'M8Z'], dtype=object)

In [110]:
df_final = pd.DataFrame()
df_final['Postcode'] = 0
df_final['Borough'] = 'string'
df_final['Neighborhood'] = 'string'
for i, code in enumerate(postcodes):
    #print(i,code)
    df_final.loc[i,'Postcode'] = code
    aux = df[ df['Postcode'] == code].Borough.to_numpy()
    df_final.loc[i,'Borough'] = aux[0]
    terms = df[df['Postcode'] == code].Neighborhood.to_numpy()
    string =''
    for term in terms:
        string = string + ', ' + term
    df_final.loc[i,'Neighborhood'] = string[2:]
df_final.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park


In [113]:
df_final.head(12)

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park
5,M9A,Queen's Park,Not assigned
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


- If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [115]:
df_final[df_final['Neighborhood']== 'Not assigned']

,Postcode,Borough,Neighborhood
5,M9A,Queen's Park,Not assigned


In [120]:
df_final.loc[5,'Neighborhood'] = df_final.loc[5,'Borough'] 

In [121]:
df_final.head(8)

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park
5,M9A,Queen's Park,Queen's Park
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North


- In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe

In [125]:
df_final.to_csv('toronto_postcode.csv')

In [124]:
print('The number of rows is ',df_final.shape[0])

The number of rows is  103
